In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import matplotlib.cm as cm
import matplotlib.dates as mdates
from scipy.signal import find_peaks
import openmeteo_requests
import requests_cache
from datetime import date, timedelta

#### Read in Data

Nur wenn "gebundelte_eco_counter_fahrradzaehler_bw.xlsx" noch nicht existiert, ansonsten skippen!

In [ ]:
# Pfad zur Excel-Datei
excel_path = "../data/gebundelte_eco_counter_fahrradzaehler_bw.xlsx"

# Excel-Datei einlesen
df = pd.read_excel(excel_path)

# Zeilen filtern, deren 'data_description' "CSV" enthält
filtered_df = df[df['data_description'].str.contains("CSV", case=False, na=False)]

# URLs extrahieren
url_list = filtered_df['data_download_url'].dropna().tolist()

# Ausgabe (optional)
#for url in url_list:
#    print(url)

general_columns = pd.read_csv(url_list[1]).columns.tolist()

# Erstelle CSV-Datei, in der Daten aller URLs gespeichert werden
all_data = pd.DataFrame()
for url in url_list:
    csv_data = pd.read_csv(url)
    assert list(csv_data.columns) == general_columns, f"Spalten stimmen nicht überein in {url}"
    all_data = pd.concat([all_data, csv_data], ignore_index=True)

# Speichere all_data lokal als CSV-Datei
all_data.to_csv("alle_fahrradzaehler_daten.csv", index=False)   
all_data.head() 

In [2]:
# Einlesen von alle_fahrradzaehler_daten.csv
data = pd.read_csv("../data/alle_fahrradzaehler_daten.csv")
data.head()

,timestamp,iso_timestamp,zählstand,stand,standort,channel_name,channel_id,counter_site,counter_site_id,domain_name,domain_id,longitude,latitude,timezone,interval,counter_serial
0,2013-01-01T01:00:00+0000,2013-01-01T01:00:00+0100,15,0,Stadt Karlsruhe,Erbprinz. West,101004165,Erbprinzenstraße,100004165,Stadt Karlsruhe,752,8.402715,49.007286,(UTC+01:00) Europe/Paris;DST,15,Y2H16070301
1,2013-01-01T02:00:00+0000,2013-01-01T02:00:00+0100,17,0,Stadt Karlsruhe,Erbprinz. West,101004165,Erbprinzenstraße,100004165,Stadt Karlsruhe,752,8.402715,49.007286,(UTC+01:00) Europe/Paris;DST,15,Y2H16070301
2,2013-01-01T03:00:00+0000,2013-01-01T03:00:00+0100,14,0,Stadt Karlsruhe,Erbprinz. West,101004165,Erbprinzenstraße,100004165,Stadt Karlsruhe,752,8.402715,49.007286,(UTC+01:00) Europe/Paris;DST,15,Y2H16070301
3,2013-01-01T04:00:00+0000,2013-01-01T04:00:00+0100,13,0,Stadt Karlsruhe,Erbprinz. West,101004165,Erbprinzenstraße,100004165,Stadt Karlsruhe,752,8.402715,49.007286,(UTC+01:00) Europe/Paris;DST,15,Y2H16070301
4,2013-01-01T05:00:00+0000,2013-01-01T05:00:00+0100,9,0,Stadt Karlsruhe,Erbprinz. West,101004165,Erbprinzenstraße,100004165,Stadt Karlsruhe,752,8.402715,49.007286,(UTC+01:00) Europe/Paris;DST,15,Y2H16070301


#### Check Data Set

TODO: Keep relevant checks for submission and clean up

In [ ]:
# Check data 
print(data.info())
#print(data.describe())
#print(data.isnull().sum())
#print(data.dtypes)

In [ ]:
# Check for duplicates
duplicates = data.duplicated()
print(f"Anzahl der Duplikate: {duplicates.sum()}")

In [ ]:
# Number of years in the dataset
data['year'] = data['timestamp'].dt.year
print(f"Jahre im Datensatz: {data['year'].nunique()} - {data['year'].unique()}")

In [ ]:
# Locations in data set
locations = data['standort'].unique()
print(f"Anzahl der Standorte: {len(locations)}")
print(f"Standorte:")
for loc in locations:
    print(loc)

Note: counter = Gerät, channel = Richtung/Messspur

In [ ]:
# Check the amount of counters per location
counters_per_location = data.groupby('standort').agg(
    anzahl_counter=('counter_serial', 'nunique'),
    counter_sites=('counter_site', lambda x: ', '.join(sorted(set(x))))
).reset_index()

print("Anzahl der Zähler pro Standort:") 
counters_per_location

In [ ]:
counters_per_location = data.groupby('standort')['counter_site_id'].nunique()
counters_per_location

In [ ]:
# Check, at which location (standort) counter_serial is null
null_serial_locations = data[data['counter_serial'].isnull()]['standort'].unique()
print(f"Standorte mit null counter_serial: {null_serial_locations}")

In [ ]:
# Check how many channels there are for a counter side
channels_per_site = (
    data.groupby(['standort', 'counter_site_id'])['channel_name']
        .nunique()
        .reset_index(name='num_channels')
)

# Kurze Übersicht
print("Beschreibung der Anzahl Channels pro Counter-Side:")
print(channels_per_site['num_channels'].describe())
print("\nBeispiele:")
print(channels_per_site.head())

# Histogramm
plt.figure(figsize=(8,4))
max_bins = channels_per_site['num_channels'].max()
plt.hist(channels_per_site['num_channels'], bins=range(1, max_bins+2), align='left', edgecolor='black')
plt.xlabel('Anzahl Channels pro Counter')
plt.ylabel('Anzahl Counter')
plt.title('Verteilung: Channels pro Counter')
plt.xticks(range(1, max_bins+1))
plt.grid(axis='y', alpha=0.3)
plt.show()

##### Check Counter

In [ ]:
# Check counter per city
city = "Stadt Heilbronn"
data_city = data[data['standort'] == city]
print(data_city["domain_id"].unique()[0])
counters = data_city[['counter_site', 'counter_site_id', 'counter_serial']].drop_duplicates().reset_index(drop=True)
tracking = data_city.groupby(['counter_site', 'counter_site_id', 'counter_serial'], dropna = False)['iso_timestamp'] \
    .agg(first_timestamp='min', last_timestamp='max') \
    .reset_index()
counters_with_tracking = counters.merge(tracking, on=['counter_site', 'counter_site_id', 'counter_serial'])
counters_with_tracking

#### Preprocess Data

In [12]:
## Clean data
data_cleaned = data.copy()

# 1. Time
# 'Isotimestamp' is local time and considers 'Sommerzeit'. Therefore, we use this for better accuracy in time representation.
# Exchange 'timestemp' with 'iso_timestamp' and convert to datetime with UTC timezone.
# Drop 'timezone' as this is identical for all entries.
# Drop 'interval' as this is equal for all entries (15 minutes).
data_cleaned['timestamp'] = pd.to_datetime(data_cleaned['iso_timestamp'], utc = True, errors='coerce') 
data_cleaned = data_cleaned.drop(columns=['iso_timestamp', 'timezone', 'interval'])

# 2. City
# Drop 'domain_name' as this is identical to 'standort'.
# Drop 'domain_id' as this is not informative.
# Rename 'standort' to 'city' for clarity.
data_cleaned = data_cleaned.rename(columns={'standort': 'city'})
data_cleaned = data_cleaned.drop(columns=['domain_name', 'domain_id'])

# 3. Counter
# Drop 'counter_serial' as this is not informative and has many missing values.
# Drop 'stand' as this is not informative.
# Rename 'counter_site' to 'counter_site_name' for clarity.
# Note: For further analysis, use 'counter_site_id' to uniquely identify counter sites.
data_cleaned = data_cleaned.rename(columns={'counter_site': 'counter_site_name'})
data_cleaned = data_cleaned.drop(columns=['counter_serial', 'stand'])

# 4. Channel
# TODO: Drop invalid channels (e.g., channel tracking cars, stand?)
# Note: For further analysis, use 'channel_id' to uniquely identify channels.

# 5. Count
# Rename 'zählstand' to 'count' for clarity.
data_cleaned = data_cleaned.rename(columns={'zählstand': 'count'})

# Save cleaned data
data_cleaned.to_csv("../data/cleaned_fahrradzaehler_daten.csv", index=False)

data_cleaned.head()

,timestamp,count,city,channel_name,channel_id,counter_site_name,counter_site_id,longitude,latitude
0,2013-01-01 00:00:00+00:00,15,Stadt Karlsruhe,Erbprinz. West,101004165,Erbprinzenstraße,100004165,8.402715,49.007286
1,2013-01-01 01:00:00+00:00,17,Stadt Karlsruhe,Erbprinz. West,101004165,Erbprinzenstraße,100004165,8.402715,49.007286
2,2013-01-01 02:00:00+00:00,14,Stadt Karlsruhe,Erbprinz. West,101004165,Erbprinzenstraße,100004165,8.402715,49.007286
3,2013-01-01 03:00:00+00:00,13,Stadt Karlsruhe,Erbprinz. West,101004165,Erbprinzenstraße,100004165,8.402715,49.007286
4,2013-01-01 04:00:00+00:00,9,Stadt Karlsruhe,Erbprinz. West,101004165,Erbprinzenstraße,100004165,8.402715,49.007286


In [16]:
## Determine median as representative value for each city and normalize data

# 1. The hourly count of a counter site is the sum of the counts of all its channels within that hour
counter_sum = (
    data_cleaned
    .groupby(["city", "counter_site_id", "timestamp"])["count"]
    .sum()
    .reset_index(name="counter_sum")
)
# 2. Compute the median count per hour for each city 
city_medians = (
    counter_sum
    .groupby(['city', 'timestamp'])['counter_sum']
    .median()
    .reset_index(name="median_count")
)

# 3. Compute the overall median count for each city & IQR
yearly_stats = (
    city_medians
    .groupby("city")["median_count"]
    .agg(
        median_count_overall="median",
        q1=lambda x: x.quantile(0.25),
        q3=lambda x: x.quantile(0.75),
    )
)
yearly_stats["IQR"] = yearly_stats["q3"] - yearly_stats["q1"]

# 4. Normalize: (x - median_count_overall) / IQR
city_norm = city_medians.merge(
    yearly_stats[["median_count_overall", "IQR"]],
    on="city",
    how="left"
)

# 5. Avoid division by zero
city_norm["normalized_count"] = (
    (city_norm["median_count"] - city_norm["median_count_overall"])
    .div(city_norm["IQR"].replace(0, pd.NA))
)
city_norm["normalized_count"] = city_norm["normalized_count"].fillna(0)

# Save normalized data
city_norm.to_csv("../data/normalized_city_fahrradzaehler_daten.csv", index=False)

city_norm.head()

,city,timestamp,median_count,median_count_overall,IQR,normalized_count
0,Landeshauptstadt Stuttgart,2013-01-01 00:00:00+00:00,5.0,20.5,48.0,-0.322917
1,Landeshauptstadt Stuttgart,2013-01-01 01:00:00+00:00,3.0,20.5,48.0,-0.364583
2,Landeshauptstadt Stuttgart,2013-01-01 02:00:00+00:00,2.0,20.5,48.0,-0.385417
3,Landeshauptstadt Stuttgart,2013-01-01 03:00:00+00:00,0.0,20.5,48.0,-0.427083
4,Landeshauptstadt Stuttgart,2013-01-01 04:00:00+00:00,2.0,20.5,48.0,-0.385417


#### Plots

##### Number counts per Location per Year

In [ ]:
# Check yearly number of counts per standort 

# Extract date from timestamp
data['date'] = data['timestamp'].dt.date

# Step 1: Get max zählstand per standort, counter_serial, and date    #TODO: warum das Maximum extrahieren statt einfach alle zählstände zusammenzuaddieren?
max_per_counter = data.groupby(['standort', 'counter_serial', 'date'])['zählstand'].max().reset_index() #TODO: lieber counter_side_id nutzen?

# Step 2: Sum these maxima per standort and date
daily_counts = max_per_counter.groupby(['standort', 'date'])['zählstand'].sum().reset_index()
daily_counts = daily_counts.rename(columns={'zählstand': 'daily_counts'})

# Step 3: Now aggregate to yearly counts
daily_counts['year'] = pd.to_datetime(daily_counts['date']).dt.year
yearly_counts = daily_counts.groupby(['standort', 'year'])['daily_counts'].sum().reset_index()
yearly_counts = yearly_counts.rename(columns={'daily_counts': 'yearly_counts'})

print("Jährliche Zählungen pro Standort:")
yearly_counts

In [ ]:
# Plot the yearly counts per standort #TODO: sind das nicht eher die Tages-Maximas aufsummiert und nicht die tatsächlichen counts?
def thousands(x, pos):
    return f'{int(x/1000)}'

standorte = yearly_counts['standort'].unique()
colors = cm.get_cmap('tab20', len(standorte))  

plt.figure(figsize=(12, 6))
for i, standort in enumerate(standorte):
    subset = yearly_counts[yearly_counts['standort'] == standort]
    plt.plot(subset['year'], subset['yearly_counts'], marker='o', 
             label=standort, color=colors(i))

plt.title("Jährliche Fahrradzählungen pro Standort")
plt.xlabel("Jahr")
plt.ylabel("Anzahl der Fahrradzählungen (in Tsd.)")

plt.gca().yaxis.set_major_formatter(FuncFormatter(thousands))
plt.gca().legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
# For Konstanz, check how many counters there are per year
konstanz_data = data[data['standort'] == 'Stadt Konstanz']
counters_per_year_konstanz = konstanz_data.groupby('year')['counter_serial'].nunique()
counters_per_year_konstanz

In [ ]:
# Plot the number of counters per year in Konstanz
plt.figure(figsize=(8, 5))
plt.plot(counters_per_year_konstanz.index, counters_per_year_konstanz.values, marker='o')
plt.title("Anzahl der Zähler pro Jahr in Konstanz")
plt.xlabel("Jahr")
plt.ylabel("Anzahl der Zähler")
plt.show()

In [ ]:
## Normalize counts per location by number of counters

# First, calculate number of active counters per standort and date i.e. counters that have non-null zählstand on that date as max value
active_counters = data.groupby(['standort', 'counter_serial', 'date'])['zählstand'].max().reset_index()
active_counters = active_counters[active_counters['zählstand'].notnull()]
active_counters = active_counters.groupby(['standort', 'date'])['counter_serial'].nunique().reset_index()
active_counters = active_counters.rename(columns={'counter_serial': 'num_active_counters'})

# Merge mit daily_counts
daily_counts = daily_counts.merge(active_counters, on=['standort', 'date'], how='left')

# Normalize daily counts
daily_counts['normalized_daily_counts'] = daily_counts['daily_counts'] / daily_counts['num_active_counters']


# Aggregate to yearly normalized counts
normalized_yearly_counts = daily_counts.groupby(['standort', 'year'])['normalized_daily_counts'].sum().reset_index()
normalized_yearly_counts = normalized_yearly_counts.rename(columns={'normalized_daily_counts': 'normalized_yearly_counts'})

print("Normalisierte jährliche Zählungen pro Standort:")
normalized_yearly_counts

In [ ]:
# Plot the normalized yearly counts per standort
def thousands(x, pos):
    return f'{int(x/1000)}'

standorte = normalized_yearly_counts['standort'].unique()
colors = cm.get_cmap('tab20', len(standorte))  # Colormap

plt.figure(figsize=(12, 6))
for i, standort in enumerate(standorte):
    subset = normalized_yearly_counts[normalized_yearly_counts['standort'] == standort]
    plt.plot(subset['year'], subset['normalized_yearly_counts'], marker='o',  # hier die Jahreswerte verwenden!
             label=standort, color=colors(i))

plt.title("Jährliche Fahrradzählungen pro Standort (normalisiert)")
plt.xlabel("Jahr")
plt.ylabel("Anzahl der Fahrradzählungen/# Aktive Tracker (in Tsd.)")

plt.gca().yaxis.set_major_formatter(FuncFormatter(thousands))
plt.gca().legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
# printe für jeden Standort die Anzahl der Datenpunkte
for location in data['standort'].unique():
    count = data[(data['standort'] == location)].shape[0]
    if count > 0:
        print(f"Standort: {location}, Anzahl der Datenpunkte: {count}")

#printe für jeden Standort alle Channel-Namen:
for location in data['standort'].unique():
    channel_names = data[data['standort'] == location]['channel_name'].unique()
    print(f"Standort: {location}, Channel Names: {channel_names}")

# printe für jeden Channel Name am Standort Stadt Freiburg die Anzahl der Datenpunkte:
for channel_name in data['channel_name'].unique():
    count = data[(data['standort'] == "Stadt Freiburg") & (data['channel_name'] == channel_name)].shape[0]
    if count > 0:
        print(f"Standort: Stadt Freiburg, Channel Name: {channel_name}, Anzahl der Datenpunkte: {count}")

"""for location in data['standort'].unique():
    for channel_name in data['channel_name'].unique():
        count = data[(data['standort'] == location) & (data['channel_name'] == channel_name)].shape[0]
        if count > 0:
            print(f"Standort: {location}, Channel Name: {channel_name}, Anzahl der Datenpunkte: {count}")
"""

In [ ]:
# prüfe, für jeden Standort, wie viele Datenpunkte es gibt 
#for location in data['standort'].unique():
location = "Stadt Freiburg"
for site_id in data['counter_site_id'].unique():
    count = data[(data['standort'] == location) & (data['counter_site_id'] == site_id)].shape[0]
    if count > 0:
        print(f"Standort: {location}, Counter Side ID: {site_id}, Anzahl der Datenpunkte: {count}")

for channel_name in data['channel_name'].unique():
    count = data[(data['standort'] == location) & (data['channel_name'] == channel_name)].shape[0]
    if count > 0:
        print(f"Standort: {location}, Channel Name: {channel_name}, Anzahl der Datenpunkte: {count}")            

In [ ]:
#printe für jeden Standort, Channel Name und Counter Site ID die Anzahl der Datenpunkte
for location in data['standort'].unique():
    data_standort = data[data['standort'] == location]
    for site_id in data_standort['counter_site_id'].unique():
        for channel_name in data_standort['channel_name'].unique():
            count = data[(data['standort'] == location) & (data['counter_site_id'] == site_id) & (data['channel_name'] == channel_name)].shape[0]
            if count > 0:
                print(f"Standort: {location}, Channel Name: {channel_name}, Counter Side ID: {site_id}, Anzahl der Datenpunkte: {count}")


In [ ]:
from datetime import datetime

# prüfe für jeden Counter (Kombination aus Standort, Channel_name und Counter_Site_ID), 
# was der jeweils früheste und späteste timestamp ist
# schreibe standort, channel_name, frühester und spätester timestamp in pandas dataframe
start_end_timestamps = pd.DataFrame(columns=['standort', 'channel_name', 'counter_site_id', 'earliest_timestamp', 'latest_timestamp'])

for location in data['standort'].unique():
    data_standort = data[data['standort'] == location]
    for site_id in data_standort['counter_site_id'].unique():
        for channel_name in data_standort['channel_name'].unique():
            subset = data[(data['standort'] == location) & (data['counter_site_id'] == site_id) & (data['channel_name'] == channel_name)]
            if not subset.empty:
                earliest = subset['timestamp'].min().replace(tzinfo=None)
                latest = subset['timestamp'].max().replace(tzinfo=None)

                start_end_timestamps = pd.concat([start_end_timestamps, pd.DataFrame([{
                    'standort': location,
                    'channel_name': channel_name,
                    'counter_site_id': site_id,
                    'earliest_timestamp': earliest,
                    'latest_timestamp': latest
                }])], ignore_index=True)                       
                
    print(location)
                
    """if earliest > datetime(2013, 1, 1, 1, 0, 0) or latest < datetime(2024, 12, 31, 23, 0, 0):
    #datetime(2023, 11, 7, 14, 30, 0)  # Jahr, Monat, Tag, Stunde, Minute, Sekunde
        print(f"{location}, {channel_name}, Frühester Timestamp: {earliest}, Spätester Timestamp: {latest}")
    else:
        print(f"{location}, {channel_name}, passt zeitlich")"""


In [ ]:
# schreibe start_end_timestamps in csv-datei
start_end_timestamps.to_csv("start_end_timestamps.csv", index=False)